In [ ]:
#########################################################
 # Projection for Reiter's Method
#########################################################
# params,solution2,transitMat,ys,Transv

function Projection_Reiter(params,solution)


        # polA_ss::Array{T,1},
        # polC_ss::Array{T,1},
        # solution::AiyagariSolution,
        # transitMat::Array{T,2},
        # ys::Array{T,1}) where {T <: Real}
        # # Ls::Array{T,1}) where {T <: Real}

        transitMat = solution.transitMat
        Transy = params.Πy

        na = params.na
        ny = params.ny
        ys = params.ys


#         # Vmat = transitMat[1:na,:]
        aGrid = params.aGrid
        aGrid[1] = 0
        Va = repeat(aGrid,ny)'*transitMat # policy rule in a a'(a)

        pol = reshape(Va,na,ny)
        Vind = zeros(Int,na*ny)
        Wind = zeros(Float64,na*ny)

        for i = 1:length(Va)
                indcc=findall(x->x<=Va[i],aGrid)
                Vind[i] = maximum(indcc)
                Wind[i] = 1-(Va[i] - aGrid[Vind[i]])/(aGrid[Vind[i]+1]-aGrid[Vind[i]])
        end

        γ =  params.γ
        β =  params.β
        R = solution.R
        w = solution.w       
        χ = params.χ    
        ϕ = params.ϕ

ls = similar(Va)  #policy rules as a function of beginning of period asset
cs = similar(Va)  #policy rules as a function of beginning of period asset

@unpack Tt, u′,l_supply,a_min,a_max,aGrid,ny,ys = params 

w = solution.w
R = solution.R

for ia = 1:na
    for iy = 1:ny
            f_c(c) = c-(R*aGrid[ia] - Va[(na)*(iy-1)+ia] + 
            w*(ys[iy]*l_supply(w,ys[iy],u′(c))) + Tt)
            cs[(na)*(iy-1)+ia] = find_zero(f_c, [a_min, a_max], atol=0.0, rtol=0.0)
            ls[(na)*(iy-1)+ia] = l_supply(w,ys[iy],u′(cs[(na)*(iy-1)+ia]))
    end
end

        ytype = repeat(kron(collect(StepRange(1, Int8(1), ny)),ones(Int,na))) #vector of individual's types

        Cpol = cs
        Lpol = ls
        Xe = (solution.w .*ys)
        Xc = 0*Xe
        Xc = kron(Xc,ones(na,1))

        resE = zeros(Float64,na*ny)
        cimp = zeros(Float64,na*ny)
        EUC = zeros(Float64,na*ny)
        cv = zeros(Float64,na*ny)

       


        for i=1:na
                for j=1:ny
                        ind = i+(j-1)*na
                        EUc = 0
                        for jp=1:ny
                                indi = Vind[ind]+(jp-1)*na
                                EUc = EUc + params.Πy'[jp,j]*( Wind[ind]*Cpol[indi] + (1-Wind[ind])*Cpol[indi+1]  )^-γ                                
                        end
                        EUC[ind] = EUc
                        resE[ind] = Cpol[ind]^-γ - β*R*EUc
                        cimp[ind] = (β*R*EUc)^(-1/γ)
                        cv[ind]  =  Cpol[ind] - cimp[ind]
                end
        end

 

#         # solution.residEuler = u′.(cs) - β*R*reshape(
#         #         solution.transitMat'*reshape(u′.(cs),na*ny,1),na,ny)
        

        return Vind,Wind,resE, Va, Cpol, Lpol
        # return Va

end


: 